1. Load Dataset

In [ ]:
import pandas as pd
import numpy as np

Dataset: Retail

In [112]:
retail_df = pd.read_csv("retail.csv")
retail_df["Products"] = retail_df["Products"].astype(str)
transactions = {} 
transactions["retail"] = [x.strip().split(",") for x in retail_df["Products"]]

Dataset: Basket

In [113]:
basket_df = pd.read_csv("basket.csv", header=None)
transactions["basket"] = (
    basket_df
    .apply(lambda x: [str(i).strip() for i in x.dropna() if str(i).strip() != ""], axis=1)
    .tolist()
)

Dataset: Groceries

In [126]:
gro_df = pd.read_csv("Groceries_transactions.csv")
gro_df["itemDescription"] = gro_df["itemDescription"].astype(str)
transactions["groceries"] = [x.strip().split(",") for x in gro_df["itemDescription"]]

Dataset: Market Basket Optimisation

In [118]:
market_df = pd.read_csv("Market_Basket_Optimisation.csv", header=None)
transactions["market"] = market_df.apply(lambda x: [i for i in x.dropna().astype(str)], axis=1).tolist()

In [119]:
for name, trans in transactions.items():
    print(f"{name} dataset:")
    print(f"  Number of transactions: {len(trans)}")
    print(f"  Avg items per transaction: {np.mean([len(t) for t in trans]):.2f}")
    print(f"  Example: {trans[0][:5]}")
    print("-"*60)

retail dataset:
  Number of transactions: 30000
  Avg items per transaction: 6.58
  Example: ['Dish Sponge', ' Flatbread with Meat', ' Chips', ' Orange', ' Butter']
------------------------------------------------------------
basket dataset:
  Number of transactions: 14963
  Avg items per transaction: 2.59
  Example: ['whole milk', 'pastry', 'salty snack']
------------------------------------------------------------
groceries dataset:
  Number of transactions: 3898
  Avg items per transaction: 8.92
  Example: ['canned beer', ' hygiene articles', ' misc. beverages', ' pastry', ' pickled vegetables']
------------------------------------------------------------
market dataset:
  Number of transactions: 7501
  Avg items per transaction: 3.91
  Example: ['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes']
------------------------------------------------------------


2. Apriori

In [120]:
import time
import pandas as pd
from apriori import runApriori
from itertools import islice

In [122]:
def subset_transactions(transactions, T):
    return list(islice(transactions, T))

def run_apriori_experiment(transactions, min_sup, min_conf):
    
    transactions_iter = (frozenset(t) for t in transactions)

    start = time.time()
    items, rules = runApriori(transactions_iter,
                              minSupport=min_sup,
                              minConfidence=min_conf)
    elapsed = time.time() - start

    num_itemsets = len(items)
    num_rules = len(rules)
    print(f"{len(transactions)} transactions processed "
          f"in {elapsed:.4f}s | {num_itemsets} frequent sets | {num_rules} rules")

    return elapsed, num_itemsets, num_rules

Retail

In [123]:
transaction_sizes = [500, 1000, 2000, 4000, 8000, 16000, 30000]
min_support = 0.01
min_confidence = 0.5


dataset_name = "retail"
transactions_all = transactions[dataset_name]

results = []

for T in transaction_sizes:
    subset = subset_transactions(transactions_all, T)
    elapsed, num_freq, num_rules = run_apriori_experiment(
        subset, min_sup=min_support, min_conf=min_confidence)
    results.append({
        "Transactions": T,
        "Time (s)": elapsed,
        "Frequent Itemsets": num_freq,
        "Rules": num_rules
    })

# Display results
results_df = pd.DataFrame(results)
print("\n Experiment Summary ")
print(results_df.to_string(index=False))

500 transactions processed in 0.7279s | 1027 frequent sets | 80 rules
1000 transactions processed in 1.0694s | 889 frequent sets | 24 rules
2000 transactions processed in 1.8376s | 878 frequent sets | 17 rules
4000 transactions processed in 4.0517s | 893 frequent sets | 33 rules
8000 transactions processed in 9.9084s | 892 frequent sets | 33 rules
16000 transactions processed in 24.9069s | 894 frequent sets | 35 rules
30000 transactions processed in 49.7681s | 896 frequent sets | 37 rules

 Experiment Summary 
 Transactions  Time (s)  Frequent Itemsets  Rules
          500  0.727930               1027     80
         1000  1.069374                889     24
         2000  1.837647                878     17
         4000  4.051708                893     33
         8000  9.908392                892     33
        16000 24.906864                894     35
        30000 49.768091                896     37


Basket

In [131]:
transaction_sizes = [500, 1000, 2000, 4000, 8000, 14963]
min_support = 0.01
min_confidence = 0.5


dataset_name = "basket"
transactions_all = transactions[dataset_name]

results = []

for T in transaction_sizes:
    subset = subset_transactions(transactions_all, T)
    elapsed, num_freq, num_rules = run_apriori_experiment(
        subset, min_sup=min_support, min_conf=min_confidence)
    results.append({
        "Transactions": T,
        "Time (s)": elapsed,
        "Frequent Itemsets": num_freq,
        "Rules": num_rules
    })

# Display results
results_df = pd.DataFrame(results)
print("\n Experiment Summary ")
print(results_df.to_string(index=False))

500 transactions processed in 0.0802s | 98 frequent sets | 1 rules
1000 transactions processed in 0.1352s | 77 frequent sets | 0 rules
2000 transactions processed in 0.3021s | 81 frequent sets | 0 rules
4000 transactions processed in 0.5157s | 69 frequent sets | 0 rules
8000 transactions processed in 1.0058s | 69 frequent sets | 0 rules
14963 transactions processed in 2.1514s | 69 frequent sets | 0 rules

 Experiment Summary 
 Transactions  Time (s)  Frequent Itemsets  Rules
          500  0.080199                 98      1
         1000  0.135221                 77      0
         2000  0.302070                 81      0
         4000  0.515723                 69      0
         8000  1.005812                 69      0
        14963  2.151399                 69      0


Groceries

In [130]:
transaction_sizes = [500, 1000, 2000, 3898]
min_support = 0.01
min_confidence = 0.5


dataset_name = "groceries"
transactions_all = transactions[dataset_name]

results = []

for T in transaction_sizes:
    subset = subset_transactions(transactions_all, T)
    elapsed, num_freq, num_rules = run_apriori_experiment(
        subset, min_sup=min_support, min_conf=min_confidence)
    results.append({
        "Transactions": T,
        "Time (s)": elapsed,
        "Frequent Itemsets": num_freq,
        "Rules": num_rules
    })

# Display results
results_df = pd.DataFrame(results)
print("\n Experiment Summary ")
print(results_df.to_string(index=False))

500 transactions processed in 4.8770s | 3978 frequent sets | 3578 rules
1000 transactions processed in 5.0747s | 2941 frequent sets | 1501 rules
2000 transactions processed in 9.3690s | 2659 frequent sets | 1151 rules
3898 transactions processed in 17.6389s | 2514 frequent sets | 955 rules

 Experiment Summary 
 Transactions  Time (s)  Frequent Itemsets  Rules
          500  4.877013               3978   3578
         1000  5.074657               2941   1501
         2000  9.369029               2659   1151
         3898 17.638914               2514    955


Market

In [51]:
transaction_sizes = [500, 1000, 2000, 4000, 7501]
min_support = 0.01
min_confidence = 0.5


dataset_name = "market"
transactions_all = transactions[dataset_name]

results = []

for T in transaction_sizes:
    subset = subset_transactions(transactions_all, T)
    elapsed, num_freq, num_rules = run_apriori_experiment(
        subset, min_sup=min_support, min_conf=min_confidence)
    results.append({
        "Transactions": T,
        "Time (s)": elapsed,
        "Frequent Itemsets": num_freq,
        "Rules": num_rules
    })

# Display results
results_df = pd.DataFrame(results)
print("\n Experiment Summary ")
print(results_df.to_string(index=False))

500 transactions processed in 0.2960s | 357 frequent sets | 112 rules
1000 transactions processed in 0.6117s | 350 frequent sets | 20 rules
2000 transactions processed in 0.9118s | 293 frequent sets | 8 rules
4000 transactions processed in 2.0938s | 302 frequent sets | 3 rules
7501 transactions processed in 3.9909s | 257 frequent sets | 2 rules

 Experiment Summary 
 Transactions  Time (s)  Frequent Itemsets  Rules
          500  0.295972                357    112
         1000  0.611733                350     20
         2000  0.911849                293      8
         4000  2.093813                302      3
         7501  3.990889                257      2


3. Brute-force approach

In [ ]:
import random, itertools, time

def brute_force_freq(transactions, items_subset):
    m = len(items_subset)
    t0 = time.time()
    counts = {}
    for r in range(1, m+1):
        for comb in itertools.combinations(items_subset, r):
            comb_set = set(comb)
            cnt = sum(1 for tr in transactions if comb_set.issubset(tr))
            counts[comb] = cnt
    t1 = time.time()
    return counts, t1-t0

In [129]:
retail_items = set()
for row in retail_df["Products"].astype(str):
    for item in row.split(","):
        retail_items.add(item.strip())

print("Retail distinct items:", len(retail_items))

basket_items = set()
for _, row in basket_df.iterrows():
    for item in row.dropna():
        basket_items.add(str(item).strip())

print("Basket distinct items:", len(basket_items))

groceries_items = set()
for row in gro_df["itemDescription"]:
    for item in row.split(","):
        groceries_items.add(item.strip())

print("Groceries distinct items:", len(groceries_items))

market_items = set()
for row in market_df[0].astype(str):
    for item in row.split(","):
        market_items.add(item.strip())

print("Market distinct items:", len(market_items))

dataset_items = {
    "retail": retail_items,
    "basket": basket_items,
    "groceries": groceries_items,
    "market": market_items
}


Retail distinct items: 40
Basket distinct items: 167
Groceries distinct items: 167
Market distinct items: 115


Retail

In [60]:
sample_items = random.sample(list(retail_items), 10)
transactions = [set(x.split(",")) for x in retail_df["Products"].astype(str)]

counts, elapsed = brute_force_freq(transactions, sample_items)
c = elapsed / (2 ** len(sample_items))
print(f"Measured elapsed={elapsed:.4f}s for m0={len(sample_items)}, c={c:.3e}")

T_est = c * (2 ** len(dataset_items['retail']))

print(f"Retail estimated brute-force time: {T_est:.2e} seconds")

transaction_sizes = [500, 1000, 2000, 4000, 8000, 16000, 30000]
c_base = c 
n_base = len(retail_df) 
m_retail = len(retail_items)

def seconds_to_readable(s):
    if s < 60: return f"{s:.2f}s"
    m = s / 60
    if m < 60: return f"{m:.2f}min"
    h = m / 60
    if h < 24: return f"{h:.2f}h"
    d = h / 24
    if d < 365: return f"{d:.2f}days"
    return f"{d/365:.2f}years"

print("Estimated Brute-force times for different transaction sizes (Retail):\n")
for n in transaction_sizes:
    c_n = c_base * (n / n_base)
    T_est = c_n * (2 ** m_retail)
    print(f"{n:>6d} transactions → {seconds_to_readable(T_est)}")

Measured elapsed=2.9179s for m0=10, c=2.850e-03
Retail estimated brute-force time: 3.13e+09 seconds
Estimated Brute-force times for different transaction sizes (Retail):

   500 transactions → 1.66years
  1000 transactions → 3.31years
  2000 transactions → 6.62years
  4000 transactions → 13.25years
  8000 transactions → 26.49years
 16000 transactions → 52.99years
 30000 transactions → 99.35years


Basket

In [61]:
sample_items = random.sample(list(basket_items), 10)
counts, elapsed = brute_force_freq(transactions, sample_items)
c = elapsed / (2 ** len(sample_items))
print(f"Measured elapsed={elapsed:.4f}s for m0={len(sample_items)}, c={c:.3e}")

m_basket = len(basket_items)
T_est = c * (2 ** m_basket)
print(f"Basket estimated brute-force time: {T_est:.2e} seconds")

transaction_sizes = [500, 1000, 2000, 4000, 8000, 14963]
c_base = c
n_base = len(basket_df)
print("\nEstimated Brute-force times for different transaction sizes (Basket):\n")
for n in transaction_sizes:
    c_n = c_base * (n / n_base)
    T_est = c_n * (2 ** m_basket)
    print(f"{n:>6d} transactions → {seconds_to_readable(T_est)}")

Measured elapsed=3.0257s for m0=10, c=2.955e-03
Basket estimated brute-force time: 5.53e+47 seconds

Estimated Brute-force times for different transaction sizes (Basket):

   500 transactions → 585699963052596560690124765197049528320.00years
  1000 transactions → 1171399926105193121380249530394099056640.00years
  2000 transactions → 2342799852210386242760499060788198113280.00years
  4000 transactions → 4685599704420772485520998121576396226560.00years
  8000 transactions → 9371199408841544971041996243152792453120.00years
 14963 transactions → 17527657094312005648397958513063389822976.00years


Groceries

In [132]:
sample_items = random.sample(list(groceries_items), 10)
counts, elapsed = brute_force_freq(transactions, sample_items)
c = elapsed / (2 ** len(sample_items))
print(f"Measured elapsed={elapsed:.4f}s for m0={len(sample_items)}, c={c:.3e}")

m_groceries = len(groceries_items)
T_est = c * (2 ** m_groceries)
print(f"Groceries estimated brute-force time: {T_est:.2e} seconds")

transaction_sizes = [500, 1000, 2000, 3898]
c_base = c
n_base = len(gro_df)
print("\nEstimated Brute-force times for different transaction sizes (Groceries):\n")
for n in transaction_sizes:
    c_n = c_base * (n / n_base)
    T_est = c_n * (2 ** m_groceries)
    print(f"{n:>6d} transactions → {seconds_to_readable(T_est)}")

Measured elapsed=0.0026s for m0=10, c=2.587e-06
Groceries estimated brute-force time: 4.84e+44 seconds

Estimated Brute-force times for different transaction sizes (Groceries):

   500 transactions → 1968448807794365879299447804827533312.00years
  1000 transactions → 3936897615588731758598895609655066624.00years
  2000 transactions → 7873795231177463517197791219310133248.00years
  3898 transactions → 15346026905564881355864485340997746688.00years


Market

In [66]:
sample_items = random.sample(list(market_items), 10)
counts, elapsed = brute_force_freq(transactions, sample_items)
c = elapsed / (2 ** len(sample_items))
print(f"Measured elapsed={elapsed:.4f}s for m0={len(sample_items)}, c={c:.3e}")
m_market = len(market_items)
T_est = c * (2 ** m_market)
print(f"Market estimated brute-force time: {T_est:.2e} seconds")

transaction_sizes = [500, 1000, 2000, 4000, 7501]
c_base = c
n_base = len(market_df)

print("\nEstimated Brute-force times for different transaction sizes (Market):\n")
for n in transaction_sizes:
    c_n = c_base * (n / n_base)
    T_est = c_n * (2 ** m_market)
    print(f"{n:>6d} transactions → {seconds_to_readable(T_est)}")

Measured elapsed=4.1285s for m0=10, c=4.032e-03
Market estimated brute-force time: 1.67e+32 seconds

Estimated Brute-force times for different transaction sizes (Market):

   500 transactions → 353989620656593315037184.00years
  1000 transactions → 707979241313186630074368.00years
  2000 transactions → 1415958482626373260148736.00years
  4000 transactions → 2831916965252746520297472.00years
  7501 transactions → 5310552289090212861050880.00years
